In [ ]:
%pip install -U 'crewai[tools]'
%pip install -U crewai

In [ ]:
from crewai import Crew, Agent, Task
import os
from dotenv import load_dotenv
from crewai_tools import SerperDevTool, ScrapeWebsiteTool

In [ ]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") or "<your_openai_key>"
SERPER_API_KEY = os.getenv("SERPER_API_KEY") or "<your_serper_key>"

In [ ]:
serper_dev_tool = SerperDevTool()
scrape_website_tool = ScrapeWebsiteTool()

class Agents:
    def researcher(self, topic):
        return Agent(
            role=f"{topic} Senior News Researcher",
            goal=f"Uncover latest news in {topic}",
            tools=[serper_dev_tool, scrape_website_tool],
            backstory=f"You're a seasoned researcher with a knack for uncovering the latest developments in {topic}. Known for your ability to find the most relevant information and present it in a clear and concise manner.",
            verbose=True,
        )

    def reporting_analyst(self, topic):
        return Agent(
            role=f"{topic} News Reporting Analyst",
            goal=f"Create detailed reports based on {topic} news analysis and research findings",
            tools=[],
            backstory=f"You're a meticulous analyst with a keen eye for detail. You're known for your ability to turn complex data into clear and concise reports, making it easy for others to understand and act on the information you provide.",
            verbose=True,
        )

In [ ]:
from textwrap import dedent

class Tasks:
    def research_task(self, agent, topic, date):
        return Task(
            description=dedent(
                f"""
                    Search news about {topic}
                    The current time is {date}
                    """
            ),
            expected_output=dedent(
                """A list of news articles about {topic} with the title, url and snippet"""
            ),
            agent=agent,
        )

    def reporting_task(self, agent):
        return Task(
            description=dedent(
                f"""
                    Review the context you got.
                    Make sure the report is detailed and contains any and all relevant information.
                """
            ),
            expected_output=dedent(
                """A fully fledge reporting of the news articles.
                Formatted as markdown without '```'"""
            ),
            agent=agent,
        )

In [ ]:
from datetime import datetime

tasks = Tasks()
agents = Agents()

# Create Agents
researcher = agents.researcher(topic="AI")
reporting_analyst = agents.reporting_analyst(topic="AI")

# Define Tasks for each agent
research_task = tasks.research_task(agent=researcher, topic="AI", date=datetime.now().strftime("%Y-%m-%d"))
reporting_task = tasks.reporting_task(agent=reporting_analyst)

# Instantiate the crew with a sequential process
crew = Crew(
    agents=[researcher, reporting_analyst],
    tasks=[
        research_task,
        reporting_task,
    ],
)

try:
    # Kick off the process
    result = crew.kickoff()
except Exception as e:
    print(e)

print("Your chuck norris joke crew is complete!")
print("Best joke: ", result)